In [5]:
import helper
import time
import numpy as np
import cv2
from PIL import Image
import random

In [6]:
def run_test(image, victim_position, fatals, victim_needs, rescue_position, rescue_resources, algorithm = 'theta*'):
    start_time = time.time()
    # WIDTH, HEIGHT = 640, 480
    # background = np.array(Image.open(image_link))
    # background = cv2.resize(background, (WIDTH, HEIGHT))
    # background = np.transpose(background, (1, 0, 2))
    # groundMapModel = helper.buildMap(background)
    # # print(groundMapModel)
    # groundMapModel = np.repeat(groundMapModel.reshape(groundMapModel.shape[0], groundMapModel.shape[1], 1), 3, axis=2)
    # groundMapModel = helper.turn_image_to_binary(groundMapModel, 1)

    res = np.array(rescue_position)
    vic = np.array(victim_position)

    if algorithm == "theta*":
        _,  cost_matrix = helper.solve_for_paths(
            image, res, vic, fatals, rescue_resources, victim_needs
        )
    elif algorithm == "a*":
        _,  cost_matrix = helper.solve_for_paths(
            image, res, vic, fatals, rescue_resources, victim_needs, algorithm="a_aristek"
        )
    elif algorithm == "BFS":
        _,  cost_matrix = helper.solve_for_path_with_BFS(
            image, res, vic, fatals, rescue_resources, victim_needs
        )
        
    def L2(p1, p2):
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2);

    s = 0
    cnt = 0
    for costs in cost_matrix:
        for cost in costs:
            if(cost != 1e9):
                cnt += 1
                s += cost
            # if path is not None:
            #     for index, point in enumerate(path):
            #         if(index < len(path) - 1):
            #             next_point = path[index + 1]
            #             s = s + L2(point, next_point)
    duration = time.time() - start_time
    return s / cnt, duration

In [7]:
WIDTH, HEIGHT = 640, 480

def getGroundMapModel(image_link: str):
    groundMapModel = np.array(Image.open(image_link))
    groundMapModel = cv2.resize(groundMapModel, (WIDTH, HEIGHT))
    groundMapModel = np.transpose(groundMapModel, (1, 0, 2))
    groundMapModel = helper.buildMap(groundMapModel)
    groundMapModel = np.repeat(
        groundMapModel.reshape(groundMapModel.shape[0], groundMapModel.shape[1], 1),
        3,
        axis=2,
    )
    groundMapModel = helper.turn_image_to_binary(groundMapModel, 1)
    return groundMapModel

groundMapModel = getGroundMapModel("test.jpg")
_, _, reachable = helper.BFS(groundMapModel, (50, 50))

def generateTest(
    groundMapModel: np.array,
    number_of_rescue_teams: int,
    number_of_victims: int,
    rescue_resources_range: int = 10,
    victim_needs_range: int = 3,
):
    # generate rescute teams' informations
    rescue_positions = []
    for _ in range(number_of_rescue_teams):
        x = random.randrange(WIDTH - 10)
        y = random.randrange(HEIGHT - 10)
        #while groundMapModel[x][y] != 1:
        while reachable[x, y] != 1:
            x = random.randrange(WIDTH - 10)
            y = random.randrange(HEIGHT - 10)
        rescue_positions.append((x, y))
    rescue_positions = np.array(rescue_positions)
    rescue_resources = np.random.randint(
        1, rescue_resources_range, size=number_of_rescue_teams
    )

    # generate victims' informations
    victim_positions = []
    for _ in range(number_of_victims):
        x = random.randrange(WIDTH - 10)
        y = random.randrange(HEIGHT - 10)
        #while groundMapModel[x][y] != 1:
        while reachable[x, y] != 1:
            x = random.randrange(WIDTH - 10)
            y = random.randrange(HEIGHT - 10)
        victim_positions.append((x, y))
    victim_positions = np.array(victim_positions)

    fatals = np.random.permutation(np.arange(1, number_of_victims + 1))
    victim_needs = np.random.randint(1, victim_needs_range, size=number_of_victims)
    return (rescue_positions, victim_positions, fatals, rescue_resources, victim_needs)

In [ ]:
for i in [8, 12, 16]:
    rescue_positions, victim_positions, fatals, rescue_resources, victim_needs = (
        generateTest(
            groundMapModel=groundMapModel, number_of_rescue_teams = i // 2, number_of_victims=i
        )
    )

    print(run_test(groundMapModel, victim_positions, fatals, victim_needs, rescue_positions, rescue_resources))

In [ ]:
rescue_positions, victim_positions, fatals, rescue_resources, victim_needs = (
    generateTest(
        groundMapModel=groundMapModel, number_of_rescue_teams = 6, number_of_victims = 12
    )
)

print(run_test(groundMapModel, victim_positions, fatals, victim_needs, rescue_positions, rescue_resources))
print(run_test(groundMapModel, victim_positions, fatals, victim_needs, rescue_positions, rescue_resources, algorithm="a*"))

(406.4633566217716, 53.577428340911865)
(399.1314532334678, 22.795886754989624)


In [ ]:
rescue_positions, victim_positions, fatals, rescue_resources, victim_needs = (
    generateTest(
        groundMapModel=groundMapModel, number_of_rescue_teams = 3, number_of_victims = 12
    )
)
print(run_test(groundMapModel, victim_positions, fatals, victim_needs, rescue_positions, rescue_resources))
print(run_test(groundMapModel, victim_positions, fatals, victim_needs, rescue_positions, rescue_resources, algorithm="a*"))
print(run_test(groundMapModel, victim_positions, fatals, victim_needs, rescue_positions, rescue_resources, algorithm="BFS"))

(440.2775370689685, 52.95148825645447)
(435.941247562051, 18.745772123336792)
(428.6020500233144, 11.098731517791748)


In [ ]:
rescue_positions, victim_positions, fatals, rescue_resources, victim_needs = (
    generateTest(
        groundMapModel=groundMapModel, number_of_rescue_teams = 8, number_of_victims = 8
    )
)
print(run_test(groundMapModel, victim_positions, fatals, victim_needs, rescue_positions, rescue_resources))
print(run_test(groundMapModel, victim_positions, fatals, victim_needs, rescue_positions, rescue_resources, algorithm="a*"))
print(run_test(groundMapModel, victim_positions, fatals, victim_needs, rescue_positions, rescue_resources, algorithm="BFS"))

(304.28189583136884, 32.35368251800537)
(301.2047888099053, 19.502323865890503)
(295.1651017506057, 32.81974673271179)
